### This notebook is meant to help determine how to setup a Markov Chain transitin matrix to reflect a desired MTTF

In [1]:
import sys
sys.path.append('..')

In [ ]:
# prompt: create a function that defines the transition matrix for a two state markov chain that must meet a specified MTTF

import numpy as np

def create_mttf_markov_chain(mttf, time_step):
  """
  Creates a 2-state Markov chain transition matrix that meets a specified MTTF.

  The two states are assumed to be 'Operational' and 'Failed'.
  The transition probabilities are defined as follows:
  P(Operational -> Operational) = p
  P(Operational -> Failed) = 1-p
  P(Failed -> Failed) = 1 (assuming failure is an absorbing state)
  P(Failed -> Operational) = 0 (assuming failure is an absorbing state)

  The Mean Time To Failure (MTTF) is the expected time to reach the Failed state
  from the Operational state. For this simple Markov chain, the MTTF is given by
  MTTF = time_step / (1-p)
  We can solve for p: p = 1 - (time_step / MTTF)

  Args:
  X
  Returns:
    A 2x2 numpy array representing the transition matrix.
    Returns None if a valid transition matrix cannot be created (e.g., mttf <= time_step).
  """
  if mttf <= time_step:
    print("Error: MTTF must be greater than the time step.")
    return None

  # Calculate the probability of staying in the operational state
  p = 1 - (time_step / mttf)

  # Define the transition matrix
  # Rows represent current state (0: Operational, 1: Failed)
  # Columns represent next state (0: Operational, 1: Failed)
  transition_matrix = np.array([
      [p, 1 - p],  # Transitions from Operational
      [0, 1]       # Transitions from Failed (absorbing state)
  ])

  return transition_matrix

In [3]:
# Example usage:
mttf_value = 100  # Desired MTTF
step_duration = 10 # Time step

transition_matrix = create_mttf_markov_chain(mttf_value, step_duration)

if transition_matrix is not None:
  print("Transition Matrix:")
  print(transition_matrix)

  # Verify MTTF (approximately, due to discrete time steps)
  # For a more accurate verification, you would simulate or use absorbing Markov chain analysis
  # Let's do a simple check of the probability of failure in one step
  prob_failure_in_one_step = transition_matrix[0, 1]
  print(f"\nProbability of failure in one step: {prob_failure_in_one_step:.4f}")
  print(f"Expected MTTF based on calculated probability: {step_duration / prob_failure_in_one_step:.2f}")

Transition Matrix:
[[0.9 0.1]
 [0.  1. ]]

Probability of failure in one step: 0.1000
Expected MTTF based on calculated probability: 100.00


In [4]:
# prompt: recreate the mttf_markov_chain function to work for n possible states

import numpy as np
def create_mttf_markov_chain_n_states(mttf, time_step, num_operational_states):
  """
  Creates an (n+1)-state Markov chain transition matrix that meets a specified MTTF.

  The states are assumed to be 'Operational_0', 'Operational_1', ...,
  'Operational_{n-1}' and 'Failed' (state n).
  Assumes transitions are only from an Operational state to the Failed state.
  All Operational states are assumed to have the same probability of transitioning
  to the Failed state per time step to achieve the desired MTTF.
  The Failed state is an absorbing state.

  The Mean Time To Failure (MTTF) from any Operational state is the expected
  time to reach the Failed state.
  MTTF = time_step / P(transition to Failed from any operational state)

  Args:
    mttf: The desired Mean Time To Failure in the same time units as time_step.
    time_step: The duration of each step in the Markov chain.
    num_operational_states: The number of operational states (n). The total
                            number of states will be n+1 (n operational + 1 failed).

  Returns:
    An (n+1)x(n+1) numpy array representing the transition matrix.
    Returns None if a valid transition matrix cannot be created (e.g., mttf <= time_step).
  """
  if mttf <= time_step:
    print("Error: MTTF must be greater than the time step.")
    return None

  if num_operational_states <= 0:
    print("Error: Number of operational states must be positive.")
    return None

  total_states = num_operational_states + 1 # Operational states + Failed state

  # Calculate the probability of transitioning to the Failed state from any
  # operational state. This probability is assumed to be the same for all
  # operational states for this simplified model to meet the single MTTF requirement.
  prob_to_failed = time_step / mttf

  if prob_to_failed > 1:
       print(f"Error: Calculated probability to failed ({prob_to_failed:.4f}) is greater than 1. "
             "Increase MTTF or decrease time_step.")
       return None

  # The probability of staying in the operational state cluster (or transitioning
  # between operational states) is 1 - prob_to_failed for each operational state.
  # For this simplified model, we assume all remaining probability is for staying
  # within the operational states (e.g., equally distributed among other operational
  # states including the current one, or just staying in the current one if no
  # transitions between operational states are modeled explicitly).
  # We'll assume for this simple model that the remaining probability is distributed
  # among the *other* operational states and potentially staying in the current one.
  # A simple way to distribute this is equally among all operational states, including the current one.
  # Let's simplify further: assume the remaining probability (1 - prob_to_failed)
  # is distributed among the 'num_operational_states' operational states.
  # A common pattern is to assume the system stays in the same operational state
  # with probability (1 - prob_to_failed) and transitions to failed with prob_to_failed.
  # Or, if states represent progressive degradation, the probability might be
  # of moving to the *next* operational state.
  # Let's implement the simple case: from any operational state, you either stay
  # in that state or transition to failed. This implies no transitions *between*
  # operational states. This still gives MTTF = time_step / prob_to_failed
  # from any operational state as long as state 0 is the starting state.

  # If states represent a sequence 0 -> 1 -> ... -> n-1 -> n (Failed),
  # then from state i (i < n-1), prob to i+1 is p, prob to n (Failed) is q.
  # From state n-1, prob to n (Failed) is r.
  # This makes the MTTF calculation more complex.

  # Let's stick to the interpretation closest to the original 2-state model:
  # there's a set of operational states, and from *any* operational state, there's
  # a probability 'q' of going to the single 'Failed' state, and probability '1-q'
  # of staying within the set of operational states (e.g., staying in the same state).
  # The MTTF from any operational state is time_step / q. So, q = time_step / mttf.

  transition_matrix = np.zeros((total_states, total_states))

  # Fill in transitions from operational states (0 to num_operational_states-1)
  prob_stay_operational = 1 - prob_to_failed

  for i in range(num_operational_states):
    # Assume the probability of staying in the *set* of operational states
    # is distributed among the operational states.
    # For simplicity, let's assume the probability of staying in the *same*
    # operational state is `prob_stay_operational`.
    transition_matrix[i, i] = prob_stay_operational
    # And the probability of transitioning to the Failed state is `prob_to_failed`.
    transition_matrix[i, num_operational_states] = prob_to_failed


  # The Failed state (index num_operational_states) is an absorbing state
  transition_matrix[num_operational_states, num_operational_states] = 1.0

  # Optional: If transitions between operational states are needed,
  # the prob_stay_operational would be distributed differently.
  # E.g., probability of moving to the next state (i+1) could be prob_stay_operational,
  # and the diagonal would be 0 for operational states except the last one.
  # This simplified model assumes no transitions between operational states explicitly.

  return transition_matrix

# Example usage for N states:
mttf_value_n = 100  # Desired MTTF
step_duration_n = 10 # Time step
num_op_states = 3    # Number of operational states

transition_matrix_n = create_mttf_markov_chain_n_states(mttf_value_n, step_duration_n, num_op_states)

if transition_matrix_n is not None:
  print(f"\nTransition Matrix for {num_op_states} operational states (Total {num_op_states+1} states):")
  print(transition_matrix_n)

  # Verify MTTF (approximately)
  # The probability of failure from any operational state in one step is the value
  # in the last column (Failed state).
  # For this matrix structure, it's the same for all operational states.
  prob_failure_in_one_step_n = transition_matrix_n[0, num_op_states] # Or any operational state row
  print(f"\nProbability of failure from any operational state in one step: {prob_failure_in_one_step_n:.4f}")
  # This should be equal to step_duration_n / mttf_value_n
  print(f"Expected MTTF based on calculated probability: {step_duration_n / prob_failure_in_one_step_n:.2f}")

# Simulate with N states:
def simulate_markov_chains_n_states(transition_matrix, num_simulations, max_steps, starting_state=0):
  """
  Simulates multiple Markov chains based on a given transition matrix.

  Args:
    transition_matrix: A numpy array representing the transition matrix.
    num_simulations: The number of independent Markov chains to simulate.
    max_steps: The maximum number of steps to simulate for each chain.
    starting_state: The initial state for the simulations (default is state 0).

  Returns:
    A list of lists, where each inner list represents the sequence of states
    for one simulation.
  """
  num_states = transition_matrix.shape[0]
  simulations = []
  failed_state_index = num_states - 1 # Assuming the last state is the failed state

  for _ in range(num_simulations):
    current_state = starting_state
    chain = [current_state]

    for _ in range(max_steps):
      # If the current state is the absorbing state (Failed), stay there
      if current_state == failed_state_index:
        chain.append(current_state)
        continue

      # Determine the probabilities for the next state based on the current state
      probabilities = transition_matrix[current_state, :]

      # Sample the next state based on the probabilities
      next_state = np.random.choice(num_states, p=probabilities)

      chain.append(next_state)
      current_state = next_state

    simulations.append(chain)

  return simulations

if transition_matrix_n is not None:
  num_simulations_n = 100
  max_simulation_steps_n = 500 # Increase max steps as MTTF is higher

  simulated_chains_n = simulate_markov_chains_n_states(transition_matrix_n, num_simulations_n, max_simulation_steps_n, starting_state=0) # Start in state 0

  print(f"\nSimulating {num_simulations_n} Markov chains with {num_op_states+1} states:")
  # You can print or further analyze the simulated_chains
  # for i, chain in enumerate(simulated_chains_n):
  #   print(f"Chain {i+1}: {chain[:20]}...") # Print first 20 steps for brevity

  # Optional: Calculate failure times for each simulation
  failure_times_n = []
  failed_state_index_n = num_op_states # The index of the failed state

  for chain in simulated_chains_n:
      # Find the first occurrence of the Failed state
      try:
          failure_step = chain.index(failed_state_index_n) * step_duration_n # Convert step index to time
          failure_times_n.append(failure_step)
      except ValueError:
          # If the chain didn't fail within max_simulation_steps
          # This simulation didn't reach the failed state
          pass # Or append max_simulation_steps * step_duration_n

  if failure_times_n:
      average_simulated_mttf_n = np.mean(failure_times_n)
      print(f"\nAverage simulated failure time across completed chains ({len(failure_times_n)} failures): {average_simulated_mttf_n:.2f}")
  else:
      print("\nNo failures observed in the simulations within the maximum steps.")



Transition Matrix for 3 operational states (Total 4 states):
[[0.9 0.  0.  0.1]
 [0.  0.9 0.  0.1]
 [0.  0.  0.9 0.1]
 [0.  0.  0.  1. ]]

Probability of failure from any operational state in one step: 0.1000
Expected MTTF based on calculated probability: 100.00

Simulating 100 Markov chains with 4 states:

Average simulated failure time across completed chains (100 failures): 113.20
